In [1]:
!pip install janome
from janome.tokenizer import Tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# Google Driveをマウント
from google.colab import drive 
drive.mount('/content/drive')
# 日本語 Wikipedia エンティティベクトルをダウンロード
!wget -P "/content/drive/My Drive/download" http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2022-09-23 06:35:12--  http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1373795477 (1.3G) [application/x-bzip2]
Saving to: ‘/content/drive/My Drive/download/20170201.tar.bz2.1’

20170201.tar.bz2.1  100%[===================>]   1.28G  11.2MB/s    in 1m 58s  

2022-09-23 06:37:11 (11.1 MB/s) - ‘/content/drive/My Drive/download/20170201.tar.bz2.1’ saved [1373795477/1373795477]

tar (child): /content/drive/My: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [11]:
!tar -jxvf "/content/drive/My Drive/download/20170201.tar.bz2"

entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


In [19]:
tkz=Tokenizer()
s="私は公園を歩く。"
ws=[w for w in  tkz.tokenize(s,wakati=True)]
ws

['私', 'は', '公園', 'を', '歩く', '。']

In [13]:
#200次元の分散表現
!pip install gensim
import gensim
from gensim.models.keyedvectors import KeyedVectors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from gensim.models.keyedvectors import KeyedVectors
model_dir = '/content/entity_vector/entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [20]:
import numpy as np
import torch
xn=torch.tensor([model[w] for w in ws])

In [21]:
xn.shape

torch.Size([6, 200])

In [22]:
xn=xn.unsqueeze(0)
xn.shape

torch.Size([1, 6, 200])

In [23]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

lstm = nn.LSTM(200,200,batch_first=True)
h0=torch.randn(1,1,200)
c0=torch.randn(1,1,200)
yn,(hn,cn)=lstm(xn,(h0,c0))

In [24]:
yn.shape

torch.Size([1, 6, 200])

In [25]:
hn.shape

torch.Size([1, 1, 200])

In [26]:
cn.shape

torch.Size([1, 1, 200])

In [28]:
labels={'名詞':0,'助詞':1,'形容詞':2,'助動詞':3,'補助記号':4,'動詞':5,'代名詞':6,'接尾辞':7,'副詞':8,'形状詞':9,'記号':10,'連体詞':11,'接頭辞':12,'接続詞':13,'感動詞':14,'空白':15}

In [29]:
import pickle
with open('dic.pkl','br') as f:
  dic=pickle.load(f)
dic['犬']

4373

In [38]:
with open('xtrain.pkl','br') as f:
  xtrain=pickle.load(f)
with open('ytrain.pkl','br') as f:
  ytrain=pickle.load(f)
xtrain[6]
ytrain[6]

[9, 0, 1, 8, 5, 7, 3, 4]

In [67]:
class MyLSTM(nn.Module):
    def __init__(self, vocsize, posn, hdim):
        super(MyLSTM, self).__init__()
        self.embd = nn.Embedding(vocsize, hdim,padding_idx=0)
        self.lstm = nn.LSTM(input_size=hdim, hidden_size=hdim,batch_first=True,
                            num_layers=2, bidirectional=True)
        self.ln   = nn.Linear(hdim*2, posn)
    def forward(self, x):
        x = self.embd(x)
        lo, (hn, cn) = self.lstm(x)
        out = self.ln(lo)
        return out

In [62]:
class MyDataset(Dataset):
    def __init__(self, xdata, ydata):
        self.data = xdata
        self.label = ydata
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.label[idx]
        return x, y

def my_collate_fn(batch):
    xdata, ydata = list(zip(*batch))
    xs = list(xdata)
    ys = list(ydata)
    return xs, ys

In [68]:
batch_size = 200
dataset = MyDataset(xdata,ydata)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)

In [69]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = MyLSTM(len(dic)+1, len(labels), 100).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

In [70]:
net.train()
for ep in range(10):
    loss10B, i = 0.0, 0
    for xs, ys in dataloader:
        xs1, ys1 = [], []
        for k in range(len(xs)):
            tid = xs[k]
            xs1.append(torch.LongTensor(tid))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid))
        xs1 = pad_sequence(xs1, batch_first=True).to(device)
        ys1 = pad_sequence(ys1, batch_first=True, padding_value=-1.0)
        output = net(xs1)
        ys1 = ys1.type(torch.LongTensor).to(device)
        loss = criterion(output[0],ys1[0])
        for h in range(1,len(ys1)):
            loss += criterion(output[h],ys1[h])
        if (i % 10 == 0):
            print(ep, i, loss10B)
            loss10B = 0.0
        else:
            loss10B += loss.item()
        i += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    outfile = "lstm1-" + str(ep) + ".model"
    torch.save(net.state_dict(),outfile)

0 0 0.0
0 10 3700.554718017578
0 20 3251.9515075683594
0 30 2788.0751342773438
0 40 2342.4857788085938
0 50 1954.811996459961
0 60 1739.6522827148438
0 70 1554.6769561767578
0 80 1429.9122314453125
0 90 1389.7843475341797
0 100 1260.2772521972656
0 110 1221.961410522461
0 120 1112.4026260375977
0 130 1067.3943405151367
0 140 1053.304557800293
0 150 952.5415954589844
0 160 919.250373840332
0 170 883.4406967163086
0 180 855.1600799560547
0 190 790.3198165893555
0 200 748.0932540893555
0 210 710.0015029907227
0 220 682.6602401733398
0 230 692.6870880126953
0 240 644.4410171508789
1 0 0.0
1 10 683.6591186523438
1 20 546.0869064331055
1 30 523.0795097351074
1 40 558.7830390930176
1 50 513.901912689209
1 60 477.1277160644531
1 70 501.50311279296875
1 80 511.1082420349121
1 90 448.84069061279297
1 100 456.19865798950195
1 110 417.3899459838867
1 120 406.0206985473633
1 130 428.4988021850586
1 140 432.1682891845703
1 150 391.97170639038086
1 160 359.2012481689453
1 170 376.26702880859375
1 180

In [52]:
net.load_state_dict(torch.load("/content/lstm0-1.model"))

<All keys matched successfully>

In [59]:
real_data_num=0
net.eval()
with torch.no_grad():
    ok = 0
    for xs, ys in dataloader:
        xs1, ys1 = [], []
        for k in range(len(xs)):
            real_data_num += len(xs[k])
            tid = xs[k]
            xs1.append(torch.LongTensor(tid))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid))
        xs1 = pad_sequence(xs1, batch_first=True).to(device)
        ys1 = pad_sequence(ys1, batch_first=True, padding_value=-1.0).to(device)
        output = net(xs1)
        ans = torch.argmax(output,dim=2)
        ok += torch.sum(ans == ys1).item()
print(ok, real_data_num, ok/real_data_num)

46300 1329489 0.034825410364433255
